# Fine-tuning BERT for multi-label text classification

# Load Dependencies

In [22]:
import pandas as pd
import numpy as np

import re
import string

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
import torch
import tensorflow as tf
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")


import logging
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", level=logging.INFO)

tensor([1.], device='mps:0')


# Load Dataset

In [3]:
dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

logging.info(f"the dataset distribution: {dataset.num_rows}")

2023-09-11 10:46:25,433 - INFO - the dataset distribution: {'train': 6838, 'test': 3259, 'validation': 886}


In [4]:
dataset["train"][0]

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

# Preprocess data

In [5]:
logging.info(f"creating a mapping table for labels")

labels = [label for label in dataset["train"].features if label not in ["ID", "Tweet"]]

label2id = {idx:label for idx, label in enumerate(labels)}
id2label = {label:idx for idx, label in enumerate(labels)}

2023-09-11 10:46:27,125 - INFO - creating a mapping table for labels


In [6]:
model_id = "bert-base-uncased"
max_length = 128
batch_size = 16
metric_name = "f1"
num_labels = len(labels)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
def text_preprocessing(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub("\#", " ", text) # remove hashtag # symbol only 
    text = re.sub(r"\\n", " ", text) # remove newlines
    text = text.lower() # text normalization
    text = re.sub("\@\w+", " ", text) # remove @
    text = re.sub('\w*\d\w*', " ", text) # remove digits
    text = re.sub("([^\x00-\x7F])+", " ", text) # remove emoji
    text = re.sub('[%s]' % re.escape(string.punctuation), " ", text) # remove punctuation
    text = re.sub("\s+", " ", text) # remove excessive white space 
    return text.strip()

In [8]:
logging.info(f"measuring text length")

train_text_len = []

for text in dataset["train"]["Tweet"]:
    processed_text = text_preprocessing(text)
    len_ = len(text.split(" "))
    train_text_len.append(len_)

logging.info(f"max length is {max(train_text_len)}")

2023-09-11 10:46:35,248 - INFO - measuring text length
2023-09-11 10:46:35,396 - INFO - max length is 58


In [9]:
examples = dataset["train"][0]
item_label = {k: examples[k] for k in examples.keys() if k in labels}
for idx, label in enumerate(labels):
    print(item_label[label])

False
True
False
False
False
False
True
False
False
False
True


In [10]:
def encoding_text(item):

    # preprocessing text
    text = [text_preprocessing(t) for t in item["Tweet"]]

    # encoding text
    encoding = tokenizer(text, 
                       padding="max_length",
                       truncation=True,
                       max_length=max_length)

    # initiate an empty matrix to store both text and labels
    labels_matrix = np.zeros((len(text), len(labels)))

    # process labels
    item_label = {label:item[label] for label in labels}

    # fill the empty array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = item_label[label]

    # converting to list
    encoding["labels"] = labels_matrix.tolist()
    
    return encoding

In [11]:
logging.info(f"preprocessing text")

encoded_dataset = dataset.map(encoding_text, batched=True, remove_columns=dataset['train'].column_names)

2023-09-11 10:46:53,026 - INFO - preprocessing text
Map: 100%|█████████████████████████████████████| 886/886 [00:00<00:00, 12855.82 examples/s]


In [12]:
print(encoded_dataset["train"][0])

{'input_ids': [101, 4737, 2003, 1037, 2091, 7909, 2006, 1037, 3291, 2017, 2089, 2196, 2031, 11830, 11527, 14354, 4105, 4737, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Model Training I: PyTorch

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                                           problem_type="multi_label_classification",
                                                           num_labels=num_labels,
                                                           id2label=id2label,
                                                           label2id=label2id
                                                          )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [35]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.322189,0.672363,0.769876,0.257336
2,0.369500,0.309953,0.694824,0.787536,0.285553
3,0.283500,0.305427,0.707035,0.798385,0.291196
4,0.245800,0.313761,0.700223,0.796378,0.267494
5,0.222900,0.311049,0.702757,0.796239,0.283296


TrainOutput(global_step=2140, training_loss=0.27597292962475356, metrics={'train_runtime': 1185.2641, 'train_samples_per_second': 28.846, 'train_steps_per_second': 1.806, 'total_flos': 2249123476753920.0, 'train_loss': 0.27597292962475356, 'epoch': 5.0})

In [38]:
trainer.evaluate()

{'eval_loss': 0.305427223443985,
 'eval_f1': 0.707035175879397,
 'eval_roc_auc': 0.7983848635714869,
 'eval_accuracy': 0.291196388261851,
 'eval_runtime': 7.9125,
 'eval_samples_per_second': 111.975,
 'eval_steps_per_second': 7.077,
 'epoch': 5.0}

# Model Training II: TensorFlow

In [48]:
# tf_dataset = encoded_dataset.with_format("tf") # not working

tf_train = encoded_dataset["train"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=["labels"],
    batch_size=batch_size,
    shuffle=True
)

tf_val = encoded_dataset["validation"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=["labels"],
    batch_size=batch_size,
    shuffle=False
)

In [17]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_id,
                                                                problem_type="multi_label_classification",
                                                                num_labels=num_labels,
                                                                id2label=id2label,
                                                                label2id=label2id
                                                               )

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print(tf_model.config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "probl

In [49]:
# prepare training: instantiate optimizer, loss and learning rate schedule 
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=2e-5) # using legacy optimizer on M1/M2 MBP
loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.CategoricalAccuracy()

tf_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [50]:
history = tf_model.fit(tf_train, 
                       validation_data=tf_val,
                       epochs=5,
                       )

Epoch 1/5


2023-09-11 14:49:11.674959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


428/428 [==============================] - ETA: 0s - loss: 0.6246 - categorical_accuracy: 0.1840

2023-09-11 14:54:04.409595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


428/428 [==============================] - 317s 707ms/step - loss: 0.6246 - categorical_accuracy: 0.1840 - val_loss: 0.4736 - val_categorical_accuracy: 0.3352
Epoch 2/5
428/428 [==============================] - 281s 653ms/step - loss: 0.4864 - categorical_accuracy: 0.2379 - val_loss: 0.4518 - val_categorical_accuracy: 0.3905
Epoch 3/5
428/428 [==============================] - 274s 639ms/step - loss: 0.4477 - categorical_accuracy: 0.3342 - val_loss: 0.3807 - val_categorical_accuracy: 0.4819
Epoch 4/5
428/428 [==============================] - 273s 637ms/step - loss: 0.4159 - categorical_accuracy: 0.4223 - val_loss: 0.3821 - val_categorical_accuracy: 0.1919
Epoch 5/5
428/428 [==============================] - 279s 651ms/step - loss: 0.4105 - categorical_accuracy: 0.3738 - val_loss: 0.4055 - val_categorical_accuracy: 0.4041


# Model Training III: TensorFlow with pre-trained BERT

reference: https://github.com/huggingface/transformers/issues/1465

In [74]:
class TFBertForMultilabelClassification(TFBertPreTrainedModel):

    def __init__(self, config, *inputs, **kwargs):
        super(TFBertForMultilabelClassification, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.bert = TFBertMainLayer(config, name='bert')
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(config.num_labels,
                                                # kernel_initializer=get_initializer(config.initializer_range),
                                                name='classifier',
                                                activation='sigmoid')

    def call(self, inputs, **kwargs):
        outputs = self.bert(inputs, **kwargs)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # logits, (hidden_states), (attentions)

In [72]:
from transformers import TFBertPreTrainedModel, TFBertMainLayer, TFBertModel

class TFBertForMultilabelClassification(TFBertPreTrainedModel):

    def __init__(self, config, *inputs, **kwargs):
        super(TFBertForMultilabelClassification, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        # self.bert = TFBertMainLayer(config, name="bert")
        self.bert = TFBertModel.from_pretrained(model_id).layers[0]
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(config.num_labels,
                                                # kernel_initializer=get_initializer(config.initializer_range),
                                                name="classifier",
                                                activation="sigmoid")

    def call(self, inputs, **kwargs):
        outputs = self.bert(inputs, **kwargs)
        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        logits = self.classifier(pooled_output)
        outputs = (logits, ) + output[2:]

        return outputs # logits, (hidden_states), (attentions)
    

In [75]:
tf_model2 = TFBertForMultilabelClassification.from_pretrained(model_id, num_labels=num_labels)

ValueError: Exception encountered when calling layer 'bert' (type TFBertMainLayer).

You have to specify either input_ids or inputs_embeds

Call arguments received by layer 'bert' (type TFBertMainLayer):
  • input_ids={}
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
# prepare training: instantiate optimizer, loss and learning rate schedule 
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=2e-5) # using legacy optimizer on M1/M2 MBP
loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.CategoricalAccuracy()

tf_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])